In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [3]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model's architecture
model.summary()


f:\Data Science\Computer Vision\env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,145,351 (4.37 MB)

 Trainable params: 1,145,351 (4.37 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Define directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=64
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=50
)

# Save the model
model.save('emotion_detection_model.h5')


Epoch 1/50


f:\Data Science\Computer Vision\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 394s 867ms/step - accuracy: 0.2371 - loss: 1.8265 - val_accuracy: 0.2564 - val_loss: 1.7877
Epoch 2/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step - accuracy: 0.2344 - loss: 1.7782 - val_accuracy: 0.2000 - val_loss: 1.7209
Epoch 3/50


C:\Users\aayya\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


448/448 ━━━━━━━━━━━━━━━━━━━━ 84s 185ms/step - accuracy: 0.2553 - loss: 1.7885 - val_accuracy: 0.2958 - val_loss: 1.7189
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 93us/step - accuracy: 0.3281 - loss: 1.7398 - val_accuracy: 0.4000 - val_loss: 1.5209
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 80s 177ms/step - accuracy: 0.2754 - loss: 1.7536 - val_accuracy: 0.3350 - val_loss: 1.6695
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 85us/step - accuracy: 0.3594 - loss: 1.7382 - val_accuracy: 0.2000 - val_loss: 1.7942
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 86s 190ms/step - accuracy: 0.2898 - loss: 1.7247 - val_accuracy: 0.3619 - val_loss: 1.5983
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 108us/step - accuracy: 0.2656 - loss: 1.7103 - val_accuracy: 0.1000 - val_loss: 1.6959
Epoch 9/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 82s 182ms/step - accuracy: 0.3198 - loss: 1.6768 - val_accuracy: 0.4242 - val_loss: 1.5214
Epoch 10/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 97us/step - accuracy: 0.3750 - loss: 1.6360 - val_accur

In [6]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


113/113 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.5390 - loss: 1.1918
Test accuracy: 54.78%
